In [674]:
import pandas as pd
import numpy as np

df = pd.read_excel('MetaTargeting.xlsx')

In [675]:
df.columns

Index(['campaign_name', 'adset_name', 'Targeting'], dtype='object')

Split Targeting by '\n'

In [676]:
tar = ['Targeting']

def remove_section(val, section):
    if pd.isnull(val):
        return val
    val = str(val)
    return val.replace(section, '').strip()


sections=['Customer List','Location:','Edit']

for section_str in sections:
    for col in tar:
        df[col] = df[col].astype(str)
        df[col] = df[col].apply(lambda x: remove_section(x, section_str))

In [677]:
sepcol = df['Targeting'].str.split('\n', expand=True)

sepcol.columns = [f'Targeting_part{i+1}' for i in range(sepcol.shape[1])]

df = df.drop(columns=['Targeting']).join(sepcol)

Split all rows containing 'Custom Audiences' to another excel

In [678]:
cust_audience = df[df['Targeting_part1'] == 'Custom Audiences']

cust_audience.to_excel('custAudience.xlsx')

In [679]:
df_nocust = df[df['Targeting_part1'] != 'Custom Audiences']

Function that deletes a cell and moves all columns to the right of the cell once to the left

In [680]:
def slide_col(start,checker):

    movcol = ['Targeting_part2', 'Targeting_part3', 'Targeting_part4', 'Targeting_part5',
        'Targeting_part6', 'Targeting_part7', 'Targeting_part8',
        'Targeting_part9', 'Targeting_part10', 'Targeting_part11',
        'Targeting_part12', 'Targeting_part13', 'Targeting_part14',
        'Targeting_part15', 'Targeting_part16', 'Targeting_part17',
        'Targeting_part18', 'Targeting_part19']
    
    
    mask = df_nocust[movcol[start]] == checker

    for i in range(start,len(movcol)-1):
        df_nocust.loc[mask, movcol[i]] = df_nocust.loc[mask, movcol[i+1]]

slide_col(0,'')

Remove 'Exclude' as a column and instead change targeting1 to 'India excl AP,KA,TM,TL'

In [681]:
slide_col(0,'Exclude ')
df_nocust.loc[df_nocust['Targeting_part2'] == 'India: Andhra Pradesh; Karnataka; Tamil Nadu; Telangana', 'Targeting_part1'] = 'India excl AP,KA,TM,TL'
slide_col(0,'India: Andhra Pradesh; Karnataka; Tamil Nadu; Telangana')
slide_col(0,'')

Removes Useless Cells

In [682]:
df_nocust['Targeting_part2'] = df_nocust['Targeting_part2'].str.replace(':','')


slide_col(1,'Age')
slide_col(1,'Age')
slide_col(2,'')
slide_col(2,'')

In [683]:
slide_col(2,'Gender')
slide_col(2,'All genders')
slide_col(2,'Detailed targeting')
slide_col(2,'Detailed targeting')
slide_col(3,'Detailed targeting')

Convert Age Ranges to a average float for easier visualizing

Change type to python and run to do above feature

Prepare Data to be recombined by shortening longer strings

In [684]:
old_df = df
df = df_nocust

combcol = ['Targeting_part4', 'Targeting_part5',
        'Targeting_part6','Targeting_part7', 'Targeting_part8',
        'Targeting_part9', 'Targeting_part10', 'Targeting_part11',
        'Targeting_part12', 'Targeting_part13', 'Targeting_part14',
        'Targeting_part15', 'Targeting_part16', 'Targeting_part17',
        'Targeting_part18', 'Targeting_part19']

In [685]:
for i in combcol:
    df.loc[df[i].str.contains('Artificial intelligence', na=False), i] = 'CSE'
    df.loc[df[i].str.contains('At high school', na=False), i] = 'Gen_HSPass'
    df.loc[df[i].str.contains('USA', na=False), i] = 'Abroad'
    df.loc[df[i].str.contains('Clark', na=False), i] = 'AvailUni'
    df.loc[df[i].str.contains('Study Abroad', na=False), i] = 'Abroad'
    df.loc[df[i].str.contains('marketing', na=False), i] = 'Business&Marketing'
    df.loc[df[i].str.contains('Admission Test', na=False), i] = 'LangTests'
    df.loc[df[i].str.contains('Yale', na=False), i] = 'EliteUni '
    df.loc[df[i].str.contains('Education Level: University graduate', na=False), i] = 'PostGrad'
    df.loc[df[i].str.contains('Interests: Master of Business', na=False), i] = 'PostGrad'

Combine all columns relating to interests and exclusions

In [686]:


df['Targeting_part7'] = df[combcol].fillna('').apply(lambda x: ' '.join(x), axis=1)
df['Targeting_part7'] = df['Targeting_part7'].str.strip()

Shorten For Easier analyzing

In [687]:
df['Targeting_part7']=df['Targeting_part7'].str.replace('Detailed targeting ','')
df['Targeting_part7']=df['Targeting_part7'].str.replace('And must also match:','and')
df['Targeting_part7']=df['Targeting_part7'].str.replace('Interests: ','')
df['Targeting_part7']=df['Targeting_part7'].str.replace('People who match: ','')
df['Targeting_part7']=df['Targeting_part7'].str.replace('Study abroad','Abroad')

Remove repeating strings

In [688]:
def remrepeat(text):
    words = text.split()
    result = []
    prev_word = None
    for word in words:
        if word != prev_word:
            result.append(word)
        prev_word = word
    return ' '.join(result)

df['Targeting_part7'] = df['Targeting_part7'].apply(lambda x: remrepeat(x))

Remove Extra Columns

In [689]:
df = df.drop(columns=['Targeting_part2','Targeting_part4', 'Targeting_part5',
        'Targeting_part6', 'Targeting_part8',
        'Targeting_part9', 'Targeting_part10', 'Targeting_part11',
        'Targeting_part12', 'Targeting_part13', 'Targeting_part14',
        'Targeting_part15', 'Targeting_part16', 'Targeting_part17',
        'Targeting_part18', 'Targeting_part19'])

Rename Targeting_part7

In [690]:
df = df.rename(columns={'Targeting_part7': 'TargetAudience'})
df = df.rename(columns={'Targeting_part1': 'Region'})
df = df.rename(columns={'Targeting_part3': 'Age Range'})

Convert DataFrame to excel as final product


In [691]:
df.to_csv('noCustAudiences.csv')

Frequently Used Functions

In [692]:
df['Age Range'].unique()

array(['25-35', None, '24-30', '24-33', '25 - 30', '25-30', '23-30',
       '24-32', '25-40'], dtype=object)

In [693]:
df.columns

Index(['campaign_name', 'adset_name', 'Region', 'Age Range', 'TargetAudience'], dtype='object')